In [1]:
# Installing the necessary packages for extracting data from the website
!pip install beautifulsoup4
!pip install requests

In [2]:
# The link for downloading the stats
link = "https://www.zupee.com/trump-cards-mania-more-information/"

In [3]:
# Importing the packages
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd

In [4]:
# Send a GET request to the web page
url = link
response = requests.get(url)
html_content = response.content

In [6]:
# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

In [7]:
# Find all the table elements on the page, because the data is stored as a table on the web page,
# you can inspect that web page to confirm
tables = soup.find_all('table')

In [9]:
# Define the headers for the CSV file or the dataframe
headers = ['Name', 'Matches', 'Runs', '6s', '4s', 'Strike Rate', 'High Score']

# Create a list to store the table data
data = []

In [10]:
# Iterate over each table
for table in tables:
    # Process the table as per your requirement
    rows = table.find_all('tr')
    for row in rows:
        columns = row.find_all('td')
        # Ignore rows that don't have the expected number of columns
        if len(columns) == len(headers):
            # Extract the text from each column
            row_data = [column.text.strip() for column in columns]
            data.append(row_data)

In [13]:
len(data)

100

In [15]:
data[:6]

[['Virat Kohli', '237', '7263', '235', '644', '130.04', '113'],
 ['Shikhar Dhawan', '217', '6617', '149', '750', '127.18', '106'],
 ['David Warner', '176', '6397', '226', '646', '139.92', '126'],
 ['Rohit Sharma', '243', '6211', '258', '554', '130.05', '109'],
 ['Suresh Raina', '205', '5528', '203', '506', '136.7', '100'],
 ['AB de Villiers', '184', '5162', '251', '413', '151.69', '133']]

In [16]:
# Saving the data as a CSV file
filename = 'zupeeData.csv'
with open(filename, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(headers)
    writer.writerows(data)

print(f"Data has been successfully stored in {filename}.")

Data has been successfully stored in zupeeData.csv.


**That's it! We have successfully downloaded the complete stats for the Trump Card Mania Game on Zupee.**

**PART 2: READING THE DATA AS CSV THEN CONVERTING IT INTO A DATAFRAME.**

In [17]:
# Let's create a dataframe to get comfortable with the data
data = pd.read_csv('/content/zupeeData.csv')

In [18]:
# View the top 5 data
data.head(5)

,Name,Matches,Runs,6s,4s,Strike Rate,High Score
0,Virat Kohli,237,7263,235,644,130.04,113
1,Shikhar Dhawan,217,6617,149,750,127.18,106
2,David Warner,176,6397,226,646,139.92,126
3,Rohit Sharma,243,6211,258,554,130.05,109
4,Suresh Raina,205,5528,203,506,136.70,100


In [19]:
data.columns

Index(['Name', 'Matches', 'Runs', '6s', '4s', 'Strike Rate', 'High Score'], dtype='object')

In [21]:
data.shape

(100, 7)

In [23]:
data['category'] = ['Gold'] * 30 + ['Silver'] * 30 + ['Bronze'] * 40

In [24]:
data.columns

Index(['Name', 'Matches', 'Runs', '6s', '4s', 'Strike Rate', 'High Score',
       'category'],
      dtype='object')

In [25]:
data.head(4)

,Name,Matches,Runs,6s,4s,Strike Rate,High Score,category
0,Virat Kohli,237,7263,235,644,130.04,113,Gold
1,Shikhar Dhawan,217,6617,149,750,127.18,106,Gold
2,David Warner,176,6397,226,646,139.92,126,Gold
3,Rohit Sharma,243,6211,258,554,130.05,109,Gold


In [49]:
data.rename(columns={'Strike Rate': 'Strike_Rate'}, inplace=True)

In [50]:
data.rename(columns={'High Score': 'High_Score'}, inplace=True)

In [51]:
data.head(2)

,Name,Matches,Runs,6s,4s,Strike_Rate,High_Score,category
0,Virat Kohli,237,7263,235,644,130.04,113,Gold
1,Shikhar Dhawan,217,6617,149,750,127.18,106,Gold


In [52]:
def rank_players(df, column_name):
    """
    Rank players based on the given column, sorting in descending order.

    Parameters:
    df (pd.DataFrame): The dataframe containing player data.
    column_name (str): The column name to sort and rank by.

    Returns:
    pd.DataFrame: A DataFrame with 'Name', the column specified, and 'Rank'.
    """
    # Sort the DataFrame by the specified column in descending order
    df_sorted = df.sort_values(by=column_name, ascending=False)

    # Add the rank column
    df_sorted['Rank'] = range(1, len(df_sorted) + 1)

    # Select the columns we want to return
    result_df = df_sorted[['Name', column_name, 'Rank']]

    return result_df

In [53]:
bronze_df = data[data['category'] == 'Bronze']

In [54]:
bronze_df.shape[0]

40

In [55]:
bronze_df.head(5)

,Name,Matches,Runs,6s,4s,Strike_Rate,High_Score,category
60,Devdutt Padikkal,57,1521,42,167,125.39,101,Bronze
61,Krunal Pandya,113,1514,56,136,133.39,86,Bronze
62,Karun Nair,76,1496,39,161,127.75,83,Bronze
63,Saurabh Tiwary,93,1494,50,111,120.10,61,Bronze
64,Marcus Stoinis,82,1478,75,111,140.63,89,Bronze


In [56]:
bronze_matches = rank_players(bronze_df, 'Matches')

In [57]:
bronze_matches.head(10)

,Name,Matches,Rank
84,Sunil Narine,162,1
66,Axar Patel,136,2
61,Krunal Pandya,113,3
72,Deepak Hooda,107,4
78,Irfan Pathan,103,5
75,Kedar Jadhav,95,6
65,Subramaniam Badrinath,95,7
99,Stuart Binny,95,8
63,Saurabh Tiwary,93,9
91,Albie Morkel,91,10


In [58]:
def save_dfs_to_excel(dfs_dict, output_file):
    """
    Saves multiple DataFrames to an Excel file with specified sheet names.

    Parameters:
    - dfs_dict (dict): A dictionary where keys are sheet names (strings) and values are DataFrames.
    - output_file (str): The path to the output Excel file. (don't put extension, just file name. Eg: output2)
    """
    with pd.ExcelWriter(f'{output_file}.xlsx') as writer:
        for sheet_name, df in dfs_dict.items():
            df.to_excel(writer, sheet_name=sheet_name, index=False)
    print(f"DataFrames have been written to {output_file}.xlsx.")

In [59]:
bronze_runs = rank_players(bronze_df, 'Runs')

In [60]:
bronze_runs.head(10)

,Name,Runs,Rank
60,Devdutt Padikkal,1521,1
61,Krunal Pandya,1514,2
62,Karun Nair,1496,3
63,Saurabh Tiwary,1494,4
64,Marcus Stoinis,1478,5
65,Subramaniam Badrinath,1441,6
66,Axar Patel,1418,7
67,Eoin Morgan,1405,8
68,Brad Hodge,1400,9
69,Sourav Ganguly,1349,10


In [61]:
# Dictionary with string keys, not DataFrames as keys
test_dict = {
    'br_matches': bronze_matches,
    'br_runs': bronze_runs
}

In [62]:
save_dfs_to_excel(test_dict, 'test_bronze')

DataFrames have been written to test_bronze.xlsx.


In [63]:
bronze_df.head(5)

,Name,Matches,Runs,6s,4s,Strike_Rate,High_Score,category
60,Devdutt Padikkal,57,1521,42,167,125.39,101,Bronze
61,Krunal Pandya,113,1514,56,136,133.39,86,Bronze
62,Karun Nair,76,1496,39,161,127.75,83,Bronze
63,Saurabh Tiwary,93,1494,50,111,120.10,61,Bronze
64,Marcus Stoinis,82,1478,75,111,140.63,89,Bronze


In [66]:
def create_ranked_dfs(df, prefix, columns, filename):
    """
    This function iterates over the specified columns and creates
    DataFrames for each column, ranking players based on that column.

    Parameters:
    - df: The input DataFrame.
    - prefix: The prefix to be added to the keys in the dictionary.
    - columns: A list of columns for which DataFrames should be created.
    - filename: The name of the output Excel file.

    Returns:
    - A dictionary where the keys are the prefix with the column name and the values are the ranked DataFrames.
    """
    ranked_dfs_dict = {}

    for column in columns:
        # Create a ranked DataFrame for each column and assign it to the dictionary
        ranked_df = rank_players(df, column)
        # Construct the key dynamically, e.g., 'br_matches', 'br_runs', etc.
        ranked_dfs_dict[f'{prefix}_{column.lower()}'] = ranked_df

    # return ranked_dfs_dict

    save_dfs_to_excel(ranked_dfs_dict, filename)

In [67]:
create_ranked_dfs(bronze_df, 'bronze', ['4s', '6s'], 'test_bronze2')

DataFrames have been written to test_bronze2.xlsx.


**BRONZE** *sheet*

In [68]:
bronze_df.columns

Index(['Name', 'Matches', 'Runs', '6s', '4s', 'Strike_Rate', 'High_Score',
       'category'],
      dtype='object')

In [69]:
create_ranked_dfs(bronze_df, 'bronze', ['Matches', 'Runs', '6s', '4s', 'Strike_Rate', 'High_Score'], 'bronze_data')

DataFrames have been written to bronze_data.xlsx.


**Silver** *sheet*

In [70]:
silver_df = data[data['category'] == 'Silver']

In [71]:
create_ranked_dfs(silver_df, 'silver', ['Matches', 'Runs', '6s', '4s', 'Strike_Rate', 'High_Score'], 'silver_data')

DataFrames have been written to silver_data.xlsx.


**Gold** *sheet*

In [72]:
gold_df = data[data['category'] == 'Gold']

In [73]:
create_ranked_dfs(bronze_df, 'gold', ['Matches', 'Runs', '6s', '4s', 'Strike_Rate', 'High_Score'], 'gold_data')

DataFrames have been written to gold_data.xlsx.
